In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Omniglot
import torchvision.models as models
from torchvision.models import resnet18
from tqdm import tqdm
#from transformers import BertModel, BertConfig
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import ConcatDataset
import torchvision.transforms
import torch.utils.data as data
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from sklearn.metrics import roc_auc_score,average_precision_score, precision_recall_curve
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Omniglot
from torchvision.models import resnet18
from tqdm import tqdm
import torch.utils.data as data
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

import os
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
#from utils import *
import logging
from torch.utils.tensorboard import SummaryWriter

import os
import numpy as np
import argparse
import configparser

In [2]:
# For on clopud result logging only

!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.9 MB/s eta 0:00:00


In [3]:
# For on clopud result logging only

!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
# Down Load MovieLens25M

!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip ml-25m.zip

--2024-05-03 17:05:57--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  14.3MB/s    in 19s     

2024-05-03 17:06:17 (13.1 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [5]:
# Down Load MovieLens2M

!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

--2024-05-03 17:06:26--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  3.41MB/s    in 1.7s    

2024-05-03 17:06:28 (3.41 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [6]:
def read_data_ml100k(data_dir, dl_name):

    names = ['user_Id', 'item_Id', 'rating', 'timestamp']

    if dl_name == "1m":
      data = pd.read_csv(data_dir,
            encoding='latin-1',
            header=None,
            engine='python',
            delimiter='::')

      print(data.shape)
      data.columns = names
      num_users = data.iloc[:,0].unique().shape[0]
      num_items = data.iloc[:,1].unique().shape[0]
      ratings = data.iloc[:2]
      time_step = data.iloc[:3]
      return data, num_users, num_items

    if dl_name == "25m":

      data = pd.read_csv(data_dir,
                        names=names, engine='python',header=0)
      print(data.shape)
      num_users = data.user_Id.unique().shape[0]
      num_items = data.item_Id.unique().shape[0]
      return data, num_users, num_items

In [7]:
def build_user_item_matrix(data):

    # Pivot table to create user-item matrix

    user_item_matrix = data.pivot_table(index="user_Id", columns="item_Id", values="rating", fill_value=0)
    return user_item_matrix

In [ ]:
data, num_users, num_items = read_data_ml100k("/content/ml-25m/ratings.csv", "25m")

In [8]:
data, num_users, num_items = read_data_ml100k("/content/ml-1m/ratings.dat", "1m")

(1000209, 4)


In [9]:
data.columns

Index(['user_Id', 'item_Id', 'rating', 'timestamp'], dtype='object')

In [10]:
from sklearn.model_selection import train_test_split


train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


print(train_data.head())
print(test_data.head())

        user_Id  item_Id  rating  timestamp
416292     2507     3035       2  974076680
683230     4087     2840       4  965431652
2434         19      457       3  978146863
688533     4118     2804       4  965804599
472584     2907      805       4  971838472
        user_Id  item_Id  rating  timestamp
895536     5412     2683       2  960243649
899739     5440      904       5  959995181
55687       368     3717       4  976311423
63727       425     1721       4  976283587
822011     4942     3697       1  962642480


In [11]:
train_matrix = train_data.pivot_table(index='user_Id', columns='item_Id', values='rating', fill_value=0)
test_matrix = test_data.pivot_table(index='user_Id', columns='item_Id', values='rating', fill_value=0)

print(train_matrix.head())
print(test_matrix.head())

item_Id  1     2     3     4     5     6     7     8     9     10    ...  \
user_Id                                                              ...   
1           5     0     0     0     0     0     0     0     0     0  ...   
2           0     0     0     0     0     0     0     0     0     0  ...   
3           0     0     0     0     0     0     0     0     0     0  ...   
4           0     0     0     0     0     0     0     0     0     0  ...   
5           0     0     0     0     0     0     0     0     0     0  ...   

item_Id  3943  3944  3945  3946  3947  3948  3949  3950  3951  3952  
user_Id                                                              
1           0     0     0     0     0     0     0     0     0     0  
2           0     0     0     0     0     0     0     0     0     0  
3           0     0     0     0     0     0     0     0     0     0  
4           0     0     0     0     0     0     0     0     0     0  
5           0     0     0     0     0     0    

In [22]:
sparsity = 1 - len(data) / (num_users * num_items)
print(f'number of users: {num_users}, number of items: {num_items}')
print(f'matrix sparsity: {sparsity:f}')

number of users: 6040, number of items: 3706
matrix sparsity: 0.955316


In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

class UserItemRatingDataset(Dataset):
    def __init__(self, user_item_matrix):

        self.ratings = torch.tensor(user_item_matrix.values)
        self.users, self.items = self.ratings.nonzero(as_tuple=True)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        user = self.users[idx]
        item = self.items[idx]
        rating = self.ratings[user, item]
        return user, item, rating


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# MF model

class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, num_factors=20):
        super(MatrixFactorization, self).__init__()
        self.user_factors = nn.Embedding(num_users, num_factors)
        self.item_factors = nn.Embedding(num_items, num_factors)


        self.user_biases = nn.Embedding(num_users, 1)
        self.item_biases = nn.Embedding(num_items, 1)

    def forward(self, user, item):


        user_embedding = self.user_factors(user)
        item_embedding = self.item_factors(item)

        # Dot product of user and item embeddings
        prediction = (user_embedding * item_embedding).sum(dim=1)

        # Add Biases
        prediction += self.user_biases(user).squeeze() + self.item_biases(item).squeeze()

        return prediction


In [23]:
import torch

class GradientDescentLineSearch(torch.optim.Optimizer):
    def __init__(self, params, lr=0.01, beta=0.5, tolerance=1e-5, max_iter=100):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if beta <= 0 or beta >= 1:
            raise ValueError("Invalid beta value: {}".format(beta))

        defaults = dict(lr=lr, beta=beta, tolerance=tolerance, max_iter=max_iter)
        super(GradientDescentLineSearch, self).__init__(params, defaults)

    def step(self, closure):

        # get initial loss and gradients
        loss = closure()
        initial_loss = loss.item()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                current_lr = group['lr']
                old_params = p.data.clone()  # Save the current value of the parameters

                for _ in range(group['max_iter']):

                    # updating the parameter with the current learning rate
                    p.data = old_params - current_lr * grad

                    optimizer.zero_grad()
                    trial_loss = closure().item()  # Compute the new loss

                    # Check the loss has reduced or not
                    if trial_loss < initial_loss - group['tolerance']:
                        initial_loss = trial_loss  # Update initial loss to new lower value
                        break  # If Good step found
                    else:
                        current_lr *= group['beta']  # Reduce learning rate
                        p.data = old_params  # Revert to original parameters if no improvement

        return loss, current_lr



In [18]:

batch_size = 1000

feature_dim = 2000

train_dataset = UserItemRatingDataset(train_matrix)
test_dataset = UserItemRatingDataset(test_matrix)

train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

model = MatrixFactorization(num_users=6040, num_items=3952, num_factors=feature_dim).to(device)

#optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay = 1e-4)

optimizer = GradientDescentLineSearch(model.parameters(), lr=0.1, beta=0.8, max_iter=50)

def closure(user, item, rating):

    optimizer.zero_grad()
    outputs = model(user, item)
    loss = criterion(outputs, rating)
    loss.backward()
    return loss

criterion = nn.MSELoss().to(device)

num_epochs = 1000

model.train()
for epoch in range(num_epochs):

    model.train()

    total_loss = 0

    for user, item, rating in train_dataloader:

        user = user.long().to(device)
        item = item.long().to(device)
        rating = rating.float().to(device)
        loss, current_lr = optimizer.step(lambda: closure(user, item, rating))
        total_loss += loss.item()


    print(f'Epoch {epoch+1}, Train Loss: {total_loss / len(train_dataloader)}, Current_Lr: {current_lr}')

    test_total_loss = 0

    with torch.no_grad():
      model.eval()
      for user, item, rating in test_dataloader:

        user = user.long().to(device)
        item = item.long().to(device)
        rating = rating.float().to(device)

        prediction = model(user, item)
        loss = criterion(prediction, rating)

        test_total_loss += loss.item()

    print(f'Epoch {epoch+1}, Test Loss: {total_loss / len(test_dataloader)}')





Epoch 1, Train Loss: 1854.8788553492705, Current_Lr: 0.1
Epoch 1, Test Loss: 7391.830662362018
Epoch 2, Train Loss: 291.23731524101953, Current_Lr: 0.1
Epoch 2, Test Loss: 1160.602435363466
Epoch 3, Train Loss: 72.90368424700142, Current_Lr: 0.1
Epoch 3, Test Loss: 290.52662229775194
Epoch 4, Train Loss: 23.1913683482919, Current_Lr: 0.1
Epoch 4, Test Loss: 92.41933356707369
Epoch 5, Train Loss: 8.643149960502406, Current_Lr: 0.1
Epoch 5, Test Loss: 34.443597603793165
Epoch 6, Train Loss: 3.6022646081879195, Current_Lr: 0.1
Epoch 6, Test Loss: 14.355293289345889
Epoch 7, Train Loss: 1.6303682834765736, Current_Lr: 0.1
Epoch 7, Test Loss: 6.497139278929032


Exception ignored in: <function _xla_gc_callback at 0x7a1eaf5b5900>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: 